In [89]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [90]:
import pandas as pd
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from src.features import build_features

warnings.filterwarnings('ignore')
sns.set_style()
plt.rcParams['figure.figsize'] = (12, 6)

In [74]:
vars_procedimientos = {f"PROCEDIMIENTO{i}": str for i in range(1, 31)}
df_procesada = pd.read_csv("../data/processed/df_procesada.csv", sep=";", dtype=vars_procedimientos)

In [84]:
df_torax = df_procesada.query("COD_HOSPITAL == 112103")

## 1. Análisis Sociodemográfico de los pacientes

En este apartado se quiere responder las siguientes preguntas:

- ¿Cuál es la **edad** de los pacientes que se realizan los procedimientos en el hospital del Tórax?
- ¿Cuál es el **sexo** de los pacientes que se realizan los procedimientos en el hospital del Tórax?
- ¿Cuál es la **previsión** de los pacientes que se realizan los procedimientos en el hospital del Tórax?
- ¿Cuál es la **letra de FONASA** de los pacientes que se realizan los procedimientos en el hospital del Tórax?

Para responder estas preguntas se utilizará la base de datos de GRD Público entre 2019 y 2021.

In [86]:
columnas_sociodemográficas = ["PREVISION", "SEXO", "RANGO_ETARIO", "Nombre Región"]

df_procesada_formato_largo = pd.melt(
    df_torax,
    id_vars=["COD_HOSPITAL", "ANIO_EGRESO", "CIP_ENCRIPTADO"] + columnas_sociodemográficas,
    value_vars=vars_procedimientos.keys(),
    value_name="procedimiento",
)

In [92]:
agrupacion_estatica = ["ANIO_EGRESO", "Nombre Región"]
agrupacion_variable = ["PREVISION", "SEXO", "RANGO_ETARIO"]
variable_a_contar = "procedimiento"

In [93]:
resultados_sociodemograficos = build_features.obtener_desglose_sociodemografico(
    df_procesada_formato_largo, agrupacion_estatica, agrupacion_variable, variable_a_contar
)

In [98]:
ruta_a_guardar = "../data/interim/sociodemografico/desglose_sociodemografico_proced.xlsx"

with pd.ExcelWriter(ruta_a_guardar) as file:
    for nombre_hoja, df_a_guardar in resultados_sociodemograficos.items():
        df_a_guardar.to_excel(file, sheet_name=nombre_hoja, index=False)